In [1]:
import datasets
import glob

In [2]:
train_dataset = {"text":[], "label":[]}

In [3]:
math_file_paths = glob.glob("../data/examination_paper_markdown" + '/' + "*.md")

for file_path in math_file_paths:
    with open(file_path, "r", encoding="utf-8") as f:
        train_dataset["text"].append(f.read())
        train_dataset["label"].append(1)

In [4]:
math_file_paths = glob.glob("../data/not_examination_paper_markdown" + '/' + "*.md")

for file_path in math_file_paths:
    with open(file_path, "r", encoding="utf-8") as f:
        train_dataset["text"].append(f.read())
        train_dataset["label"].append(0)

In [5]:
train_dataset = datasets.Dataset.from_dict(train_dataset)

In [6]:
train_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 2779
})

In [7]:
test_dataset = {"text":[], "label":[]}

In [8]:
math_file_paths = glob.glob("../data/test_examination_paper_markdown" + '/' + "*.md")

for file_path in math_file_paths:
    with open(file_path, "r", encoding="utf-8") as f:
        test_dataset["text"].append(f.read())
        test_dataset["label"].append(1)

In [9]:
math_file_paths = glob.glob("../data/test_not_examination_paper_markdown" + '/' + "*.md")

for file_path in math_file_paths:
    with open(file_path, "r", encoding="utf-8") as f:
        test_dataset["text"].append(f.read())
        test_dataset["label"].append(0)

In [10]:
test_dataset = datasets.Dataset.from_dict(test_dataset)

In [11]:
datadict = datasets.DatasetDict({
    "train":train_dataset,
    "test":test_dataset
})

In [196]:
datadict.save_to_disk("./dataset")

Saving the dataset (0/1 shards):   0%|          | 0/2779 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/387 [00:00<?, ? examples/s]

# 新增

In [2]:
import sys
import os

sys.path.append('../')

from paper_markdown_text_classifier import extract_text

In [4]:
datadict = datasets.load_dataset("ranWang/test_paper_textClassifier")

Found cached dataset parquet (/home/ran/.cache/huggingface/datasets/ranWang___parquet/ranWang--test_paper_textClassifier-7f659cfaa72da6d9/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
train_dataset = datadict["train"]

In [6]:
train_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 2779
})

# 因为原先数据集中没有保存文件地址，现在使得新增的数据中可以添加文件地址(这个文件地址仅用于自己debug使用)

In [7]:
empty_column_data = [''] * len(train_dataset['label'])

In [8]:
train_dataset = train_dataset.add_column('file_path', empty_column_data) 

In [9]:
train_dataset

Dataset({
    features: ['text', 'label', 'file_path'],
    num_rows: 2779
})

In [10]:
from pathlib import Path
from tqdm import tqdm
import os


counterexamples = Path("./negative_file").glob("**/*.doc*")

column = {"text":[], "label":[], "file_path": []}
for file_path in tqdm(list(counterexamples)):
    _, ext = os.path.splitext(file_path)
    try:
        text = extract_text(str(file_path), ext)
    except:
        continue
        
    column["text"].append(text)
    column["label"].append(0)
    column["file_path"].append(str(file_path))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5629/5629 [06:14<00:00, 15.04it/s]


In [11]:
from pathlib import Path

counterexamples = Path("./positive_file").glob("**/*.doc*")

for file_path in tqdm(list(counterexamples)):
    _, ext = os.path.splitext(file_path)
    try:
        text = extract_text(str(file_path), ext)
    except:
        continue
        
    column["text"].append(text)
    column["label"].append(1)
    column["file_path"].append(str(file_path))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1128/1128 [01:13<00:00, 15.28it/s]


In [12]:
train_dataset = datasets.concatenate_datasets([train_dataset, datasets.Dataset.from_dict(column)])

In [13]:
train_dataset

Dataset({
    features: ['text', 'label', 'file_path'],
    num_rows: 9536
})

In [14]:
datadict["train"] = train_dataset

In [15]:
datadict

DatasetDict({
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 387
    })
    train: Dataset({
        features: ['text', 'label', 'file_path'],
        num_rows: 9536
    })
})

In [17]:
datadict["test"] = datadict["test"].add_column('file_path', [''] * len(datadict["test"]['label'])) 

In [18]:
datadict.save_to_disk("./new_classification_datasets")

Saving the dataset (0/1 shards):   0%|          | 0/387 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/9536 [00:00<?, ? examples/s]